In [7]:
!pip install imbalanced-learn scikit-learn pandas
import pandas as pd
from imblearn.over_sampling import SMOTE
from google.colab import files
import numpy as np

# CSV 파일 업로드
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# 데이터 불러오기 (인코딩 지정)
df = pd.read_csv(filename, encoding='utf-8')

# 데이터 구조 확인
print(df['label'].value_counts())

#SMOTE 적용
X = df.drop('label', axis=1)
y = df['label']
smote = SMOTE(random_state=42, k_neighbors=2)
X_resampled, y_resampled = smote.fit_resample(X, y)

# SMOTE 결과 확인
print(pd.Series(y_resampled).value_counts())

# 데이터프레임으로 합치기
df_resampled = pd.DataFrame(X_resampled, columns=X.columns)
df_resampled['label'] = y_resampled

# 증강된 데이터 CSV로 저장 (UTF-8)
df_resampled.to_csv('resampled_data.csv', encoding='utf-8', index=False)

# 데이터 로드
df = pd.read_csv('resampled_data.csv')

# 'label' 열을 제외한 수치형 열만 선택
numeric_cols = df.columns.drop('label')

# 각 범주별로 증강할 샘플 수 설정
augmentation_factors = {
    'normal': 1000,
    'abnormal': 1000
}

augmented_data_list = []

# 각 범주별로 데이터 증강
for label, factor in augmentation_factors.items():
    subset = df[df['label'] == label]

    # 원본 데이터는 항상 포함
    augmented_subset_list = [subset]

    # 증강할 데이터 수 계산
    num_to_augment = factor - subset.shape[0]

    if num_to_augment > 0:
        for _ in range(num_to_augment):
            original_sample = subset.sample(n=1, replace=True)

            # 노이즈 생성
            noise = np.random.normal(0, subset[numeric_cols].std() * 0.05, size=original_sample[numeric_cols].shape)

            # 원본 데이터에 노이즈를 더해 새로운 데이터 생성
            new_data = original_sample.copy()
            new_data[numeric_cols] += noise

            augmented_subset_list.append(new_data)

    augmented_subset = pd.concat(augmented_subset_list, ignore_index=True)
    augmented_data_list.append(augmented_subset)

# 모든 증강된 데이터를 하나로 합치기
final_df = pd.concat(augmented_data_list, ignore_index=True)

# 음수 값이 하나라도 있는 행을 제거
final_df = final_df[(final_df[numeric_cols] >= 0).all(axis=1)]

# '최종회전수(rot)'와 'Stall최대값' 열 반올림
if 'Final_Rotation_rot' in final_df.columns:
    final_df['Final_Rotation_rot'] = final_df['Final_Rotation_rot'].round()
if 'Stall_Max_Value' in final_df.columns:
    final_df['Stall_Max_Value'] = final_df['Stall_Max_Value'].round()

# 최종 데이터프레임을 새로운 CSV 파일로 저장
final_df.to_csv('resampled_data_final_cleaned.csv', index=False)
# 최종 데이터 프레임의 label값 확인
print(final_df['label'].value_counts())


Saving summary.csv to summary (5).csv
label
abnormal    17
normal       4
Name: count, dtype: int64
label
normal      17
abnormal    17
Name: count, dtype: int64
label
normal      1000
abnormal     952
Name: count, dtype: int64
